<a href="https://colab.research.google.com/github/quynhu-d/hse_risk_management_1/blob/main/notebooks/RM_part4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Проект

## Библиотеки

In [ ]:
import pandas as pd
import numpy as np

import datetime
from datetime import datetime

from tqdm.auto import tqdm, trange
from copy import copy, deepcopy

import yfinance as yf

import sys
import warnings
warnings.simplefilter("ignore")
from tqdm.auto import tqdm


import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
sns.set_style('whitegrid')

In [ ]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from statsmodels.api import OLS

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder = '/content/drive/MyDrive/RM_project/'

In [ ]:
random_seed = 42
np.random.RandomState(seed=random_seed)

RandomState(MT19937) at 0x7FD70B30D240

In [ ]:
from stoch_models import *

## Загрузка данных

In [ ]:
data = pd.read_excel(folder+'Final_data.xlsx')
# data = pd.read_excel('./Final_data.xlsx')

In [ ]:
print(data.shape, '\n')

print(data.columns, '\n')

(2922, 32) 

Index(['Date', 'Курс доллара', 'Курс евро', 'Нефть', 'МосБиржа', 'РТС',
       'Золото', 'Серебро', 'Платина', 'КБД 0.25', 'КБД 0.5', 'КБД 1', 'КБД 5',
       'КБД 10', 'КБД 15', 'КБД 20', 'КБД 30', 'ОФЗ_26207', 'ОФЗ_26215',
       'ОФЗ_26218', 'ОФЗ_26219', 'ОФЗ_26222', 'Аэрофлот', 'Газпром',
       'ГМК Норникель', 'Лукойл', 'МТС', 'ПИК', 'Роснефть', 'Сбербанк',
       'Татнефть', 'Яндекс'],
      dtype='object') 



In [ ]:
# Заполнение пропусков
data.fillna(method='bfill', inplace=True)
data.fillna(method='ffill', inplace=True)

### Разбиение данных

In [ ]:
def split_data(df, split_date, start_date=None, end_date=None):
    train = df[df.Date <= split_date]
    test = df[df.Date > split_date]
    if start_date is not None:
        train = train[train.Date >= start_date]
    if end_date is not None:
        test = test[test.Date <= end_date]
    return train, test

In [ ]:
#разбиваем данные на train и test по дате

df_train, df_test = split_data(data, split_date='2022-06-09')
print(df_train.shape[0], df_test.shape[0])

2717 205


## Часть 4: оценка справедливой стоимости

Портфель состоит из инструментов.\
Инструменты:
1. Облигации (5 шт. по 10 млн.руб.)
2. Акции (10 шт. по 1 млн. руб.)
3. Валюта (доллар и евро, по 100 млн. руб.)

Риск-факторы влияют на стоимость инструментов.

In [ ]:
def make_predictions_LR(X_train, y_train, X_test):
    model = LinearRegression(fit_intercept=False)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return y_pred

In [ ]:
class Fair_value_measurement:
    def __init__(self, target_name, factors_name,
                 df, last_date,
                 n_steps=10, N_traj=100,
                 metric='mape'):

        self.target_name = target_name
        self.risk_factors = factors_name
        self.df = df
        self.train, self.test = split_data(df, split_date=last_date)
        self.n_steps = n_steps
        self.N_traj = N_traj

        self.metric = metric

        self.factors_sim = self.make_simulations_for_risk_factors(self.metric)

    def make_simulations_for_risk_factors(self, metric='mape'):
        # симулируется n траекторий для каждого риск-фактора на t шагов вперед
        dt = np.diff(list(self.df.index))
        dW_corr = generate_dW_corr(self.df[self.risk_factors], dt=dt, w0=0, N_traj=self.N_traj)

        df_res = {}
        for factor, dWn in tqdm(zip(self.risk_factors, dW_corr), total=len(self.risk_factors)):
            sm = Stoch_Models(factor_name=factor,
                  value_train = self.train[factor].values, value_test = self.test[factor].values,
                  t_train = list(self.train.index), t_test = list(self.test.index),
                  models=['m1', 'm2', 'm3'], N_traj=self.N_traj, dW_N=dWn)
            sm.choose_model(metric=metric, plot_all=0, plot_best=0, print_metric=0)
            df_res[factor] = sm.future_simulation(n_steps=self.n_steps, plot=0)

        return df_res

    def plot_predictions(self, fig_size=(18, 6)):
        fig, axs = plt.subplots(1, 1, figsize=fig_size)
        axs.plot(self.train.index, self.simulations[m].T, color=self.colors[m], linewidth=0.5)
        axs.plot(self.t_train, self.val_train, color=self.colors['factor'], linewidth=1.5, label='factor')
        axs.legend()
        axs.set_title(self.name)
        plt.show()



    def make_predictions(self):
        x_train = self.train[self.risk_factors]
        y_train = self.train[self.target_name]
        y_test = self.test[self.target_name]

        y_pred_s = []
        for i in range(0, self.N_traj):
            # формируем x_test из симуляций риск-факторов
            x_test = pd.DataFrame(index=self.test.index[:self.n_steps])
            for factor in self.factors_sim.keys():
                x_test[factor] = self.factors_sim[factor][i]
            y_pred = make_predictions_LR(x_train, y_train, x_test)
            y_pred_s.append(np.array(y_pred))

        self.predictions = np.array(y_pred_s)

### Облигации

In [ ]:
risk_factors_1 = ['Курс доллара', 'Курс евро', 'Нефть', 'МосБиржа', 'РТС',
       'Золото', 'Серебро', 'Платина', 'КБД 0.25', 'КБД 1', 'КБД 5', 'КБД 10', 'КБД 20']

print(len(risk_factors_1))

13


In [ ]:
bonds = ['ОФЗ_26207', 'ОФЗ_26215', 'ОФЗ_26218', 'ОФЗ_26219', 'ОФЗ_26222']

### Акции

In [ ]:
stocks = ['Аэрофлот', 'Газпром', 'ГМК Норникель', 'Лукойл', 'МТС', 'ПИК', 'Роснефть', 'Сбербанк', 'Татнефть', 'Яндекс']

In [ ]:
FV_sber = Fair_value_measurement(target_name='Сбербанк', factors_name = risk_factors_1, df=data, last_date= '2022-06-11')

  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
FV_sber.make_predictions()

[array([117.667699  , 116.18609548, 116.8948989 , 112.00678509,
       111.67119317, 109.2255894 , 109.33371324, 111.65956959,
       109.23216601, 109.77692024]), array([117.667699  , 121.89762589, 119.51686595, 120.73151897,
       119.89770834, 115.63127331, 113.93929422, 110.7342465 ,
       113.27289564, 111.23945371]), array([117.667699  , 115.37974027, 112.82868846, 118.81631309,
       121.17353747, 127.56913953, 130.83751826, 127.93878976,
       128.79515661, 129.39800561]), array([117.667699  , 118.62043118, 120.40245721, 118.65229985,
       118.87883645, 117.62348555, 118.80585682, 121.73406714,
       123.26458463, 121.32665136]), array([117.667699  , 117.65763137, 115.7257318 , 114.33748438,
       113.00657216, 114.54645177, 113.61641806, 120.16193081,
       118.0462378 , 114.66675427]), array([117.667699  , 115.92023568, 115.46336188, 118.15078424,
       119.61084812, 117.89896962, 117.92777612, 117.75633407,
       118.26808445, 118.76772157]), array([117.667699  , 

### Валюта

In [ ]:
risk_factors_2 = ['Нефть', 'МосБиржа', 'РТС', 'Золото', 'Серебро', 'Платина', 'КБД 0.25', 'КБД 1', 'КБД 5', 'КБД 10', 'КБД 20']